# Training 
Use this file to train networks where mu is fixed. classesmu.py is modified from its original version by Yi Hong Teoh, so that one need just specify one value for $\mu$. 

In [17]:
import sys
sys.path.insert(0, '/Users/marinadrygala/Desktop/Marina/')
import classesmu
from classesmu import BatchSimulatedSpinLattice as bsslmu
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
import time
import ic_functions
from ic_functions import chain, circle
import neural_nets
from neural_nets import Net

## Instructions
To train a network on a given dataset enter the file path for the desired file under the variable data_file, as well as the number of training examples from that file you wish to train with under the variable data_size.

In [18]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

"""Specify the path where the data is stored. The required items from the file are then stored are variables."""

data_file = '/Users/marinadrygala/Desktop/Marina/mu_fixed/Data/Data_N=4_Epsillon=0.27_Size=180984.pickle'
d = pickle.load(open(data_file, "rb"))
ic = d['ic']
m=d['m']
mu=d['mu']
N = ic.n
test_set = d['test']


"""The size of the training set should be specified by the parameter data_size"""
data_size = 25000

In [19]:
"""Creating a dataset for pytorch"""
class Chain_Data(Dataset):
    'Characterizes an ionchain dataset for PyTorch'
    
    def __init__(self):
        'Initialization'
        self.train_inputs = d['inputs'][:data_size]
        self.train_outputs = d['outputs'][:data_size]
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.train_inputs)
 

    def __getitem__(self, index):
        'Generates one sample of data'
        return (self.train_inputs[index],self.train_outputs[index])


## Instructions C'td
Change the batch_size parameter as desired.

In [20]:
batch_size = 100

training_data = Chain_Data()
training_loader = DataLoader(dataset=training_data, batch_size=batch_size,
                             shuffle=True)



## Instructions C'td
Specify the desired size for the hidden layer  as well as the number of epochs if training for a fixed number of epochs. Can also uncomment and modify the section below where it says 'Loading previous model for more training' to continue training on a saved model.

In [21]:
"""Specify network architecture. The Net class is imported from neural_nets. If the model inputs are the Jijs and the
outputs the Omegas then the sizes of the input and output layers is fixed."""
input_size = int(N*(N-1)/2)
output_size = N*m
hidden_size = 128*(N-2)
model = Net(input_size, hidden_size, output_size).to(device)
model.double()
"""Specify epochs parameter if training for a set number of epochs."""
epochs = 50

"""Specify the optimizer used as well as the model if a previously saved model will be used loaded for more training"""
optimizer = torch.optim.Adam(model.parameters())

##Loading previous model for more training
# path = '/home/gherkin/Marina/N=3/test_1/Epoch_32.pt'
# model = Net(input_size, hidden_size, output_size)
# checkpoint = torch.load(path)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# for state in optimizer.state.values():
#     for k, v in state.items():
#         if isinstance(v, torch.Tensor):
#             state[k] = v.double()

"""Specify the loss function being used."""
criterion = nn.MSELoss()


model.double()
model = model.to(device)


## Instructions C'td

Option 1:
Train for a fixed number of epochs

Option 2:
Train until the error on the test set is below a desired threshold value. Where the error on the test set is defined as the percentage difference between the true $J_{i,j}$ value and the on given by the network predictions.

In [23]:
t=time.time()
losses=[]
errors=[]

##Option 1:
# for epoch in range(epochs):
#     model.train()
#     for batch, (Jij, Omega) in enumerate(training_loader):
#         optimizer.zero_grad()
#         Omega_pred = model(Jij).view(batch_size,N,m)
#         Jij_pred = bsslmu(ic, mu, Omega_pred, dev=device).normalize()
#         loss = criterion(Jij, Jij_pred)
#         #loss = criterion(Omega, model(Jij).float())
#         loss.backward()
#         optimizer.step()
#     losses.append(loss.item())
    
#     model.eval()
#     Omega_pred = model(test_set).view(test_set.size()[0],N,m)
#     Jij_pred = bsslmu(ic, mu, Omega_pred,dev=device).normalize()
#     Error = torch.mean(torch.abs(test_set-Jij_pred))*100
#     errors.append(Error.item())
#     print ('Epoch [{}], Loss: {:.4f}, Test Error: {}'.format(epoch+1, loss.item(), round(Error.item(),4)))
# elapsed_time = time.time()-t
# print('Total Time: {}'.format(elapsed_time))

##Option 2
# epoch=0
Error = float('inf')
while Error>0.25:
    model.train()
    for batch, (Jij, Omega) in enumerate(training_loader):
        optimizer.zero_grad()
        Omega_pred = model(Jij).view(batch_size,N,m)
        Jij_pred = bsslmu(ic, mu, Omega_pred, dev=device).normalize()
        loss = criterion(Jij, Jij_pred)
        #loss = criterion(Omega, model(Jij).float())
        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    
    
    model.eval()
    Omega_pred = model(test_set).view(test_set.size()[0],N,m)
    Jij_pred = bsslmu(ic, mu, Omega_pred,dev=device).normalize()
    Error = torch.mean(torch.abs(test_set-Jij_pred))*100
    errors.append(Error.item())
    
    print ('Epoch [{}], Loss: {:.4f}, Test Error: {}'.format(epoch+1, loss.item(), round(Error.item(),4)))
    epoch+=1
elapsed_time = time.time()-t
print('Total Time: {}'.format(elapsed_time))

Epoch [1], Loss: 538.2577, Test Error: 44.6185
Epoch [2], Loss: 543.7800, Test Error: 41.9366
Epoch [3], Loss: 518.3958, Test Error: 45.1956
Epoch [4], Loss: 554.7560, Test Error: 50.5803
Epoch [5], Loss: 566.5651, Test Error: 37.6129
Epoch [6], Loss: 514.4138, Test Error: 44.236
Epoch [7], Loss: 544.8864, Test Error: 48.3504
Epoch [8], Loss: 538.2360, Test Error: 35.2897
Epoch [9], Loss: 529.4975, Test Error: 51.0636
Epoch [10], Loss: 535.1819, Test Error: 46.0832
Epoch [11], Loss: 528.9104, Test Error: 52.9685
Epoch [12], Loss: 535.6420, Test Error: 26.8858
Epoch [13], Loss: 531.5771, Test Error: 35.9331
Epoch [14], Loss: 534.0310, Test Error: 35.2995
Epoch [15], Loss: 536.7645, Test Error: 47.2922
Epoch [16], Loss: 545.9149, Test Error: 39.2714
Epoch [17], Loss: 535.2150, Test Error: 43.8037
Epoch [18], Loss: 531.2017, Test Error: 47.6329
Epoch [19], Loss: 517.2136, Test Error: 42.5098
Epoch [20], Loss: 560.7607, Test Error: 37.6097
Epoch [21], Loss: 533.0706, Test Error: 44.4565
Ep

In [24]:
"""It is often helpful to visualize the network outputs, as well as the intented Jijs and those produced by the
predictions. This code works for after network is trained."""
print('Test Set:\n',test_set)
print('Jijs give by Omegas predicted for test set:\n', Jij_pred)
print('Mean test set error:\n', Error.item())


Test Set:
 tensor([[0.5774, 0.0000, 0.0000, 0.5774, 0.0000, 0.5774],
        [0.5000, 0.0000, 0.5000, 0.5000, 0.0000, 0.5000],
        [0.4472, 0.0000, 0.4472, 0.4472, 0.4472, 0.4472],
        [0.4472, 0.4472, 0.4472, 0.4472, 0.0000, 0.4472]],
       dtype=torch.float64)
Jijs give by Omegas predicted for test set:
 tensor([[-0.3395, -0.1931,  0.0414,  0.8937,  0.1580, -0.1483],
        [-0.0891, -0.1810,  0.3138,  0.1406, -0.7595, -0.5139],
        [ 0.1013,  0.1168,  0.1838,  0.6960,  0.2418,  0.6320],
        [ 0.3534, -0.0078, -0.2068,  0.1213, -0.8989, -0.0979]],
       dtype=torch.float64, grad_fn=<ThMulBackward>)
Mean test set error:
 40.74359564774515


## Instructions C'td
Input the desired file path for which to save the trained network to into the variable named saved_data_file.

In [ ]:
saved_data_file = '/Users/marinadrygala/Desktop/Marina/mu_fixed/other_cost/N={}/m={}_Data_Size_{}_Error_{}_Time_{}_Epochs_{}.pt'.format(n,m,data_size,round(Error.item(),4),round(elapsed_time,2),epoch+1)

d['epoch'] = epoch
d['model_state_dict'] = model.state_dict()
d['optimizer_state_dict'] = optimizer.state_dict()                                                                                                                                    
d['losses'] = losses
d['errors'] = errors
d['input_size'] = input_size
d['hidden_size'] = hidden_size
d['output_size'] = output_size
                                                                                                                                    
torch.save(d, saved_data_file)